In [78]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [79]:
 import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import _target_encoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier

import pandas as pd
from category_encoders import OneHotEncoder, TargetEncoder

import warnings

# 경고 무시
warnings.filterwarnings("ignore", category=FutureWarning)

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 예시로 'NanumGothic'을 사용

In [80]:
import random
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [81]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [82]:
train = train.drop(['ID'], axis=1)
test = test.drop(['ID'], axis=1)

In [83]:
train

,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,Married,...,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,Separated,...,Native,US,US,US,Single,0,0,0,Under Median,0
2,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,Married,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860
3,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other,Divorced,...,Native,US,US,US,Single,0,0,0,Under Median,850
4,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other,Divorced,...,Native,US,US,US,Head of Household (HOH),0,0,0,Unknown,570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,33,M,High graduate,Children or Armed Forces,52,Manufacturing (Durable),Handlers/Cleaners,White,All other,Single,...,Native,US,US,US,Single,0,0,0,Under Median,1300
19996,20,F,College,Full-Time,12,Education,Admin Support (include Clerical),White,Mexican-American,Single,...,Native,US,Mexico,Mexico,Nonfiler,0,0,0,Under Median,850
19997,22,M,College,Children or Armed Forces,52,Transportation,Technicians & Support,White,All other,Single,...,Native,US,US,US,Single,0,0,0,Unknown,999
19998,76,F,High graduate,Not Working,0,Not in universe or children,Unknown,White,All other,Widowed,...,Native,US,Scotland,England,Single,0,0,0,Under Median,0


In [84]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     20000 non-null  int64 
 1   Gender                  20000 non-null  object
 2   Education_Status        20000 non-null  object
 3   Employment_Status       20000 non-null  object
 4   Working_Week (Yearly)   20000 non-null  int64 
 5   Industry_Status         20000 non-null  object
 6   Occupation_Status       20000 non-null  object
 7   Race                    20000 non-null  object
 8   Hispanic_Origin         20000 non-null  object
 9   Martial_Status          20000 non-null  object
 10  Household_Status        20000 non-null  object
 11  Household_Summary       20000 non-null  object
 12  Citizenship             20000 non-null  object
 13  Birth_Country           20000 non-null  object
 14  Birth_Country (Father)  20000 non-null  object
 15  Bi

In [85]:
test['Household_Status'].fillna('Nonfamily householder', inplace=True)

In [86]:
# 원-핫 인코딩을 적용할 열 지정
one_hot_cols = ['Gender']
# one_hot_cols = ['Gender', 'Income_Status','Tax_Status','Citizenship','Industry_Status','Occupation_Status']
# 원-핫 인코더 생성 및 train 데이터에 적용
one_hot_encoder = OneHotEncoder(cols=one_hot_cols, use_cat_names=True)
train_encoded = one_hot_encoder.fit_transform(train[one_hot_cols])

# 동일한 원-핫 인코더를 test 데이터에 적용
test_encoded = one_hot_encoder.transform(test[one_hot_cols])

In [87]:
# # 레이블 인코더 객체 생성
# label_encoder = LabelEncoder()
# 
# # one_hot_cols에 명시된 열을 제외한 카테고리형 변수를 인코딩 대상으로 설정
# encoding_target = [col for col in train.columns if train[col].dtype == 'object' and col not in one_hot_cols]
# 
# # 인코딩 대상 열에 대해 레이블 인코딩 적용
# for col in encoding_target:
#     # train과 test 데이터셋에서 해당 열의 모든 값을 문자열로 변환
#     test_encoded[col] = train[col].astype(str)
#     test_encoded[col] = test[col].astype(str)
# 
#     # 레이블 인코더를 train 데이터에 적합시키고 변환
#     label_encoder.fit(train[col])
#     train_encoded[col] = label_encoder.transform(train[col])
# 
#     # test 데이터에도 동일한 인코더를 적용
#     test_encoded[col] = label_encoder.transform(test[col])

In [88]:

# 타겟 인코더 객체 생성
target_encoder = TargetEncoder()

# one_hot_cols에 명시된 열을 제외한 카테고리형 변수를 인코딩 대상으로 설정
encoding_target = [col for col in train.columns if train[col].dtype == 'object' and col not in one_hot_cols]

# 인코딩 대상 열에 대해 타겟 인코딩 적용
for col in encoding_target:
    # 타겟 인코더를 train 데이터의 해당 열과 타겟에 적합시키고 train 데이터 변환
    train_encoded[col] = target_encoder.fit_transform(train[col], train['Income'])

    # test 데이터에는 적합시킨 인코더를 이용해 변환만 적용
    test_encoded[col] = target_encoder.transform(test[col])

In [89]:
train_encoded = pd.concat([train_encoded, train[['Working_Week (Yearly)','Age','Gains','Losses','Income']]], axis=1)
test_encoded = pd.concat([test_encoded, test[['Working_Week (Yearly)','Age','Gains','Losses']]], axis=1)

train = train_encoded
test = test_encoded

In [90]:
train = train.drop(train[train['Gains'] > 60000].index)
train = train.drop(train[train['Income'] > 8000].index)

# train['working_week_0'] = (train['Working_Week (Yearly)']==0).astype(int)
# test['working_week_0'] = (test['Working_Week (Yearly)']==0).astype(int)
# 
# train['working_week_1'] = ((train['Working_Week (Yearly)']>0) & (train['Working_Week (Yearly)']<10)).astype(int)
# test['working_week_1'] = ((train['Working_Week (Yearly)']>0) & (train['Working_Week (Yearly)']<10)).astype(int)
# 
# 
# train['working_week_3'] = ((train['Working_Week (Yearly)']>=45) & (train['Working_Week (Yearly)']<52)).astype(int)
# test['working_week_3'] = ((train['Working_Week (Yearly)']>=45) & (train['Working_Week (Yearly)']<52)).astype(int)
# 
# train['working_week_4']=(train['Working_Week (Yearly)']==52).astype(int)
# test['working_week_4']=(test['Working_Week (Yearly)']==52).astype(int)

In [91]:
train

,Gender_M,Gender_F,Education_Status,Employment_Status,Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,Household_Status,...,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Income_Status,Working_Week (Yearly),Age,Gains,Losses,Income
0,1,0,346.883212,746.928051,608.765668,512.062257,563.554230,574.993697,687.201172,728.328241,...,560.650659,574.009177,571.103049,60.947341,545.810820,4,63,0,0,425
1,1,0,867.269906,746.928051,703.643885,512.062257,563.554230,574.993697,632.185780,625.933469,...,560.650659,574.009177,571.103049,615.808365,537.030294,52,37,0,0,0
2,0,1,664.652140,746.928051,739.130826,750.374677,512.730914,574.993697,687.201172,728.328241,...,560.650659,574.009177,571.103049,742.472403,537.030294,52,58,3411,0,860
3,1,0,664.652140,746.928051,527.744681,1002.417563,563.554230,574.993697,692.336803,625.933469,...,560.650659,574.009177,571.103049,615.808365,537.030294,52,44,0,0,850
4,0,1,664.652140,746.928051,527.744681,488.010638,563.554230,574.993697,692.336803,728.328241,...,560.650659,574.009177,571.103049,676.317995,545.810820,52,37,0,0,570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,0,664.652140,488.226979,907.412063,655.152927,563.554230,574.993697,368.479404,625.933469,...,560.650659,574.009177,571.103049,615.808365,537.030294,52,33,0,0,1300
19996,0,1,651.468108,746.928051,616.393852,750.374677,563.554230,420.946483,368.479404,505.561828,...,560.650659,381.119527,381.285041,60.947341,537.030294,12,20,0,0,850
19997,1,0,651.468108,488.226979,977.865801,1002.417563,563.554230,574.993697,368.479404,452.012766,...,560.650659,574.009177,571.103049,615.808365,545.810820,52,22,0,0,999
19998,0,1,664.652140,0.000000,0.000000,0.000000,563.554230,574.993697,324.719577,625.933469,...,560.650659,443.515864,677.862674,615.808365,537.030294,0,76,0,0,0


In [94]:
test

,Gender_M,Gender_F,Education_Status,Employment_Status,Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,Household_Status,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Income_Status,Working_Week (Yearly),Age,Gains,Losses
0,1,0,483.034146,488.226979,0.000000,0.000000,563.554230,574.993697,368.479404,625.933469,...,560.650659,560.650659,495.955250,532.154739,615.808365,537.030294,0,79,0,0
1,1,0,325.116719,488.226979,0.000000,0.000000,563.554230,381.213933,368.479404,505.561828,...,560.650659,560.650659,574.009177,571.103049,60.947341,537.030294,0,47,0,0
2,0,1,483.034146,488.226979,527.744681,512.062257,563.554230,574.993697,368.479404,505.561828,...,560.650659,560.650659,574.009177,571.103049,615.808365,537.030294,52,18,0,0
3,0,1,867.269906,746.928051,718.083532,512.062257,563.554230,574.993697,687.201172,620.124322,...,560.650659,560.650659,574.009177,571.103049,742.472403,545.810820,30,39,0,0
4,1,0,0.000000,488.226979,0.000000,0.000000,563.554230,420.946483,368.479404,63.779775,...,560.650659,560.650659,574.009177,571.103049,60.947341,545.810820,0,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,0,699.367076,488.226979,747.366876,1002.417563,563.554230,574.993697,368.479404,625.933469,...,560.650659,560.650659,574.009177,571.103049,615.808365,537.030294,52,31,0,0
9996,1,0,651.468108,488.226979,527.744681,655.152927,563.554230,574.993697,687.201172,728.328241,...,560.650659,560.650659,574.009177,571.103049,742.472403,537.030294,52,27,0,0
9997,1,0,664.652140,488.226979,527.744681,512.062257,512.730914,574.993697,368.479404,505.561828,...,560.650659,560.650659,574.009177,571.103049,60.947341,537.030294,7,18,0,0
9998,1,0,0.000000,488.226979,0.000000,0.000000,563.554230,574.993697,368.479404,63.779775,...,560.650659,560.650659,574.009177,571.103049,60.947341,537.030294,0,9,0,0


In [95]:
# train['Income'] = np.log1p(train['Income'])

In [96]:
X = train[train.columns.drop('Income')]
Y = train['Income']

In [97]:
# X와 Y로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

In [102]:
import optuna
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

def objective(trial):
    # 모델 선택
    # model_type = trial.suggest_categorical('model_type', ['xgboost', 'lightgbm', 'random_forest', 'gradient_boosting'])
    model_type = 'catboost'
    # 모델 별 파라미터 설정
    if model_type == 'xgboost':
        param = {
            'objective': 'reg:squarederror',
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 9),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 9),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            # L1 정규화 매개변수 (alpha)
            'alpha': trial.suggest_float('alpha', 0.0, 1.0),
            # L2 정규화 매개변수 (lambda)
            'lambda': trial.suggest_float('lambda', 1.0, 100.0),
        }
        model = XGBRegressor(**param)
    elif model_type == 'lightgbm':
        param = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', -1, 9),
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        }
        model = LGBMRegressor(**param)
    elif model_type == 'random_forest':
        param = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 9),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 8),
        }
        model = RandomForestRegressor(**param)
    elif model_type == 'gradient_boosting':
        param = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 9),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 8),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        }
        model = GradientBoostingRegressor(**param)

    elif model_type == 'catboost':
        param = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'depth': trial.suggest_int('depth', 4, 10),
            'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
            'border_count': trial.suggest_int('border_count', 1, 255),
            'logging_level': 'Silent',
        }
        model = CatBoostRegressor(**param)

    # 모델 학습
    model.fit(X_train, y_train)

    # 예측 및 평가
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    return rmse

# Optuna 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

# 최적의 파라미터 출력
print(f"Best trial: {study.best_trial.params}")

[I 2024-04-03 21:44:51,894] A new study created in memory with name: no-name-9d3e9b6a-bf03-403b-8a09-18f0809f1e3f
[I 2024-04-03 21:44:53,890] Trial 0 finished with value: 543.4112691917474 and parameters: {'learning_rate': 0.0728001054517817, 'n_estimators': 660, 'depth': 4, 'l2_leaf_reg': 4, 'border_count': 214}. Best is trial 0 with value: 543.4112691917474.
[I 2024-04-03 21:44:58,016] Trial 1 finished with value: 548.7079342684171 and parameters: {'learning_rate': 0.1578510474175081, 'n_estimators': 778, 'depth': 8, 'l2_leaf_reg': 9, 'border_count': 95}. Best is trial 0 with value: 543.4112691917474.
[I 2024-04-03 21:44:58,539] Trial 2 finished with value: 544.7474007521162 and parameters: {'learning_rate': 0.22733864240025778, 'n_estimators': 246, 'depth': 4, 'l2_leaf_reg': 7, 'border_count': 206}. Best is trial 0 with value: 543.4112691917474.
[I 2024-04-03 21:44:59,139] Trial 3 finished with value: 541.3771072607667 and parameters: {'learning_rate': 0.28090321129230744, 'n_estima

KeyboardInterrupt: 

In [99]:
#  # 최적의 파라미터로 모델 재학습
# best_params = study.best_trial.params
# model = XGBRegressor(**best_params)
# model.fit(X_train, y_train)
# 
# # 특성 중요도 추출
# feature_importances = model.feature_importances_
# 
# # 특성 이름을 가져오기 (X_train이 pandas DataFrame일 경우)
# feature_names = X_train.columns
# 
# # 특성 중요도를 DataFrame으로 변환
# importance_df = pd.DataFrame({
#     'Feature': feature_names,
#     'Importance': feature_importances
# }).sort_values(by='Importance', ascending=False)
# 
# # 중요도가 높은 순으로 특성 출력
# print(importance_df)
# 
# # 특성 중요도 시각화
# plt.figure(figsize=(10, 8))
# plt.barh(importance_df['Feature'], importance_df['Importance'])
# plt.xlabel('Importance')
# plt.ylabel('Feature')
# plt.title('Feature Importance')
# plt.gca().invert_yaxis()  # 중요도가 높은 특성을 위로
# plt.show()

In [99]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


best_params = study.best_trial.params
xgb_model = XGBRegressor(**best_params)
xgb_model.fit(X_train, y_train)

# Best trial: {'learning_rate': 0.02550289892498271, 'n_estimators': 382, 'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.8120163205571489, 'colsample_bytree': 0.6287152249249107}

# 테스트 데이터에 대한 예측 수행
y_pred_xgb = xgb_model.predict(X_test)
# RMSE 계산
rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
print(f"XGB RMSE: {rmse_xgb}")

XGB RMSE: 537.2666897761321


In [101]:
best_params = study.best_trial.params
lgbm_model = LGBMRegressor(**best_params)
lgbm_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred_lgbm = lgbm_model.predict(X_test)
# RMSE 계산
rmse_lgbm = mean_squared_error(y_test, y_pred_lgbm, squared=False)
print(f"lgbm RMSE: {rmse_lgbm}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000473 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 536
[LightGBM] [Info] Number of data points in the train set: 15967, number of used features: 21
[LightGBM] [Info] Start training from score 552.680904
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

In [103]:
best_params = study.best_trial.params
cat_model = CatBoostRegressor(**best_params)
cat_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred_cat = cat_model.predict(X_test)
# RMSE 계산
rmse_cat = mean_squared_error(y_test, y_pred_cat, squared=False)
print(f"cat RMSE: {rmse_cat}")

0:	learn: 665.1445419	total: 12.3ms	remaining: 7.17s
1:	learn: 658.9514072	total: 22.5ms	remaining: 6.58s
2:	learn: 652.8485167	total: 34.1ms	remaining: 6.62s
3:	learn: 647.3036989	total: 45ms	remaining: 6.55s
4:	learn: 641.9979369	total: 55.6ms	remaining: 6.46s
5:	learn: 636.8110203	total: 66.9ms	remaining: 6.47s
6:	learn: 632.0203196	total: 79ms	remaining: 6.54s
7:	learn: 627.4058268	total: 85.5ms	remaining: 6.18s
8:	learn: 623.1013476	total: 96.6ms	remaining: 6.19s
9:	learn: 619.0404642	total: 108ms	remaining: 6.2s
10:	learn: 615.2483985	total: 118ms	remaining: 6.17s
11:	learn: 611.6297735	total: 130ms	remaining: 6.2s
12:	learn: 608.1987995	total: 141ms	remaining: 6.2s
13:	learn: 604.9297040	total: 151ms	remaining: 6.18s
14:	learn: 602.0513434	total: 162ms	remaining: 6.16s
15:	learn: 599.2084686	total: 172ms	remaining: 6.13s
16:	learn: 596.5381681	total: 183ms	remaining: 6.12s
17:	learn: 594.1022426	total: 193ms	remaining: 6.09s
18:	learn: 591.7430593	total: 203ms	remaining: 6.06s
1

In [104]:
final_pred = (y_pred_xgb + y_pred_lgbm+y_pred_cat )/3

final_pred[final_pred<100] = 0

In [105]:
final_pred

array([910.9977906 , 611.97180012,   0.        , ..., 853.09560826,
         0.        , 886.60217871])

In [106]:
rmse_xgb = mean_squared_error(y_test, final_pred, squared=False)
rmse_xgb

536.5749985501996

In [107]:
pred_final_xgb = xgb_model.predict(test)
# pred_final_xgb = np.expm1(pred_final_xgb)
# pred_final_xgb[pred_final_xgb<100] = 0

pred_final_cat = cat_model.predict(test)
# pred_final = np.expm1(pred_final)
# pred_final_cat[pred_final_cat<100] = 0

pred_final_lgbm = lgbm_model.predict(test)
# pred_final = np.expm1(pred_final)
# pred_final_lgbm[pred_final_lgbm<100] = 0
# 
# pred_final_dt = dt_model.predict(test)
# # pred_final = np.expm1(pred_final)
# pred_final_dt[pred_final_dt<100] = 0

In [108]:
import numpy as np

# pred_final_xgb, pred_final_cat, pred_final_lgbm은 NumPy 배열로 가정
# 이 배열들은 각 모델의 예측값을 담고 있음

# 세 모델의 예측값을 평균내어 최종 예측값 계산
pred_ensemble = (pred_final_xgb +  pred_final_lgbm+ pred_final_cat) / 3
pred_ensemble[pred_ensemble<100] = 0


In [115]:
submission = pd.read_csv("../data/sample_submission.csv")
pred_final_cat[pred_final_cat<100] = 0
submission['Income'] = pred_final_cat
submission

,ID,Income
0,TEST_0000,0.000000
1,TEST_0001,0.000000
2,TEST_0002,443.869962
3,TEST_0003,587.714966
4,TEST_0004,0.000000
...,...,...
9995,TEST_9995,933.756089
9996,TEST_9996,654.530351
9997,TEST_9997,469.184484
9998,TEST_9998,0.000000


In [116]:
submission.to_csv("../submission/test27.csv", index = False)